## Check Mediastack Dataset

#### Metadatata

In [8]:
import pandas as pd
import re

In [9]:
import sys
sys.path.append('../utils')
import functions

In [ ]:
mediastack_df = pd.read_csv('../data/raw/news_sentiment_analysis.csv')

mediastack_df.head()

In [ ]:
functions.show_basic_info(mediastack_df)

In [ ]:
functions.show_data_types(mediastack_df)

In [ ]:
functions.show_missing_values(mediastack_df)